In [29]:

import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *

from tensorflow.keras.activations import *

In [34]:
def make_model():
    
    dropout = 0.5     
    recurrent_dropout = 0.1
    num_outputs = 3 
    overall_lenght = 13

    
    input_sequence = Input(shape=(overall_lenght,),name='input')
    
    rnn = Bidirectional(LSTM(300,recurrent_dropout=recurrent_dropout,dropout=dropout,return_sequences=True))
    
    output = rnn(input_sequence)
    
    pred = Dense(num_outputs,name='predicitions')
    drop = Dropout(dropout)
    
    output = output(pred)
    output = output(drop)
    
    output_prob = Activation('softmax',name='final')(output)
    
    model = Model(inputs= input_sequence, outputs = output_prob,name='probs')
    
    model.summary()
    return model
    

In [35]:
model = make_model()



ValueError: Input 0 of layer bidirectional_11 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 13]